In [2]:
import requests
import numpy as np
import pandas as pd
import re
from pathlib import Path
import nltk
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/saisandeshk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
class NaiveBayesClassifier:
    def __init__(self, min_freq=2):
        """
        Initialize the Naive Bayes classifier.

        Args:
            min_freq (int): Minimum frequency threshold for a word to be included in vocabulary.
                           Words appearing less than min_freq times will be treated as UNK token.
                           Default: 1 (include all words)

        Attributes:
            class_probs (dict): P(class) for each class
                Example: {0: 0.5, 1: 0.5}

            word_probs (dict): P(word|class) for each word and class
                Example: {
                    0: {'hello': 0.5, 'world': 0.4, '<UNK>': 0.1},
                    1: {'hello': 0.3, 'world': 0.5, '<UNK>': 0.2}
                }

            vocabulary (dict): Word to index mapping, including special UNK token
                Example: {'<UNK>': 0, 'hello': 1, 'world': 2}

            min_freq (int): Minimum frequency threshold for vocabulary inclusion
                Example: If min_freq=2, words must appear at least twice to be included

        Note:
            - Words appearing less than min_freq times in training data will be mapped to <UNK>
            - <UNK> token is automatically added to vocabulary as first token (index 0)
            - Probability for <UNK> is calculated during training based on rare words
        """
        self.class_probs = None
        self.word_probs = None
        self.vocabulary = None
        self.min_freq = min_freq

    def preprocess_text(self, text):
        """
        Preprocess the input text by converting to lowercase, removing non-word characters,
        and filtering out common stop words.

        Args:
            text (str): Raw input text
                Example: "Hello, World! How are you doing today?"

        Returns:
            list: List of cleaned, tokenized, and filtered words with stop words removed
                Example: ['hello', 'world', 'doing', 'today']

        Note:
            - Converts all text to lowercase
            - Removes punctuation and special characters
            - Splits text into individual tokens
            - Removes common English stop words (e.g., 'a', 'an', 'the', 'is', 'are', 'how')
            - Stop words are removed using NLTK's English stop words list
        """
        # Import stop words from NLTK
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('english'))

        # Convert to lowercase
        text = text.lower()

        # Extract word characters only and split into tokens
        tokens = re.findall(r'\w+', text)

        # Remove stop words
        filtered_tokens = [token for token in tokens if token not in stop_words]

        return filtered_tokens

    def create_vocabulary(self, texts):
        """
        Create vocabulary from training texts by mapping unique words to indices,
        considering minimum frequency threshold and adding UNK token.

        Args:
            texts (list): List of text documents
                Example: [
                    "Hello world hello",
                    "Hello there",
                    "World is beautiful"
                ]

        Returns:
            dict: Mapping of words to unique indices, including UNK token
                Example (with min_freq=2): {
                    '<UNK>': 0,    # Special token for rare/unseen words
                    'hello': 1,    # Frequency=3, included in vocab
                    'world': 2,    # Frequency=2, included in vocab
                    # 'there' and 'beautiful' not included (frequency=1 < min_freq=2)
                }

        Note:
            - Always includes <UNK> token at index 0
            - Only includes words that appear >= min_freq times
            - Word frequency is counted across all documents
            - Uses preprocess_text function for preprocessing
            - Words below frequency threshold will be mapped to UNK during feature extraction
        """

        # BEGIN CODE : naive_bayes.create_vocabulary
        class_vocab = {}
        min_freq = self.min_freq
        for text in texts:
            words = self.preprocess_text(text)
            for word in words:
                if word in class_vocab:
                    class_vocab[word] += 1
                else:
                    class_vocab[word] = 1
        # print(class_vocab)
        vocabulary = {'<UNK>': 0}
        index = 1
        for word in class_vocab:
            if class_vocab[word] >= min_freq:
                vocabulary[word] = index
                index += 1
        return vocabulary
    def extract_features(self, texts, vocabulary):
        """
        Convert texts to bag-of-words feature vectors using the vocabulary,
        where each element represents the count of word occurrences (not binary presence/absence).

        Args:
            texts (list): List of text documents
                Example: ["hello world hello", "world is beautiful"]
            vocabulary (dict): Word to index mapping with UNK token
                Example: {'<UNK>': 0, 'hello': 1, 'world': 2}

        Returns:
            np.array: Feature matrix where each row is a document vector
                Example: For the above input with min_freq=2:
                array([
                    [0, 2, 1],  # First doc: 0 UNKs, 2 'hello's, 1 'world'
                    [2, 0, 1]   # Second doc: 2 UNKs (one each for 'is' and 'beautiful'), 0 'hello's, 1 'world',
                ])

        Note:
            - Each row represents one document
            - Each column represents the count of a specific word
            - First column is always UNK token count
            - Words not in vocabulary are counted as UNK
            - Shape of output: (n_documents, len(vocabulary))
            - Uses preprocess_text function for preprocessing
        """

        # BEGIN CODE : naive_bayes.extract_features
        # class_vocab = {}
        # min_freq = self.min_freq
        # for text in texts:
        #     words = self.preprocess_text(text)
        #     for word in words:
        #         if word in class_vocab:
        #             class_vocab[word] += 1
        #         else:
        #             class_vocab[word] = 1
        array = np.array([[0 for i in range(len(vocabulary))] for j in range(len(texts))])
        for i in range(len(texts)):
            words = self.preprocess_text(texts[i])
            for word in words:
                if word not in vocabulary :
                    array[i][0] += 1
                else:
                    array[i][vocabulary[word]] += 1
        return array 
    def calculate_class_probabilities(self, y):
        """
        Estimate probability P(class) for each class from training labels.

        Args:
            y (list): List of class labels
                Example: [0, 0, 1, 1, 0, 1]

        Returns:
            dict: Estimated probability for each class
                Example: {
                    0: 0.5,    # 3 out of 6 samples are class 0
                    1: 0.5     # 3 out of 6 samples are class 1
                }

        Note:
            - Probabilities sum to 1 across all classes
            - Handles any number of unique classes
        """
        # BEGIN CODE : naive_bayes.extract_features
        array = y
        class_probs = {}
        for i in range(len(array)):
            if array[i] in class_probs:
                class_probs[array[i]] += 1
            else:
                class_probs[array[i]] = 1
        for key in class_probs:
            class_probs[key] /= len(array)
        return class_probs
    def calculate_word_probabilities(self, X, y, vocabulary, alpha=1.0):
        """
        Calculate conditional probability P(word|class) for each word and class,
        including probability for UNK token.

        Args:
            X (np.array): Document-term matrix (with UNK counts in first column)
                Example: array([
                    [0, 2, 1],  # Document 1: 0 UNKs, 2 of word 1, 1 of word 2
                    [1, 0, 1],  # Document 2: 1 UNK, 0 of word 1, 1 of word 2
                ])
            y (list): Class labels
                Example: [0, 1]
            vocabulary (dict): Word to index mapping with UNK token
                Example: {'<UNK>': 0, 'hello': 1, 'world': 2}
            alpha (float): Laplace smoothing parameter, default=1.0

        Returns:
            dict: Nested dict with P(word|class) for each word and class
                Example: {
                    0: {
                        '<UNK>': 0.167,    # P(word=UNK|class=0)
                        'hello': 0.5,     # P(word='hello'|class=0)
                        'world': 0.333      # P(word='world'|class=0)
                    },
                    1: {
                        '<UNK>': 0.4,    # P(word=UNK|class=1)
                        'hello': 0.2,     # P(word='hello'|class=1)
                        'world': 0.4      # P(word='world'|class=1)
                    }
                }

        Note:
            - Uses Laplace smoothing to handle unseen words
            - UNK token probability is learned from training data
            - Formula: P(word|class) = (count(word,class) + α) / (total_words_in_class + α|V|)
            - |V| is vocabulary size (including UNK token)
        """

        # BEGIN CODE : naive_bayes.calculate_word_probabilities
        V = len(vocabulary)
        classes = set(y)  # unique class labels
        word_probs = {c: {} for c in classes}

        for c in classes:
            # get indices of documents that belong to class c
            class_indices = [idx for idx, label in enumerate(y) if label == c]

            # sum counts for all words in these documents
            total_words = 0
            word_counts = [0] * V
            for idx in class_indices:
                for w in range(V):
                    word_counts[w] += X[idx][w]
                total_words += sum(X[idx])

            # compute probabilities with Laplace smoothing
            for word, w_idx in vocabulary.items():
                word_probs[c][word] = (word_counts[w_idx] + alpha) / (total_words + alpha * V)

        return word_probs
    def predict(self, X_text):
        """
        Predict classes for new documents using Naive Bayes algorithm,
        handling unknown words using UNK token.

        Args:
            X_text (list): List of text documents
                Example: [
                    "hello world",
                    "beautiful day"  # 'day' is unknown, treated as UNK
                ]

        Returns:
            list: Predicted class labels
                Example: [0, 1]

        Theory:
            The standard Naive Bayes formula for text classification is:
            P(class|document) ∝ P(class) * ∏ P(word|class)

            For unknown words not in vocabulary:
            - They are mapped to UNK token
            - P(UNK|class) is used in probability calculation

            We use log space to prevent numerical underflow:
            log(P(class|document)) ∝ log(P(class)) + Σ log(P(word|class))

        Implementation:
            For each document:
            1. Preprocess and tokenize text
            2. Replace unknown words with UNK token
            3. Calculate log probabilities using appropriate word or UNK probabilities
            4. Return class with highest log probability score

        Note:
            - Uses preprocess_text function for preprocessing
            - Words not in vocabulary are treated as UNK token
            - UNK probability is used for out-of-vocabulary words
        """
        # BEGIN CODE : naive_bayes.predict
        predicted_labels = []
        y = [0 for i in range(len(X_text))]
        vocabulary = self.create_vocabulary(X_text)
        X = self.extract_features(X_text, vocabulary)
        class_probs = self.calculate_class_probabilities(y)
        word_probs = self.calculate_word_probabilities(X, y, vocabulary)
        for text in X_text:
            words = self.preprocess_text(text)
            word_indices = [vocabulary[word] if word in vocabulary else 0 for word in words]
            class_scores = {}
            for c in class_probs:
                class_scores[c] = np.log(class_probs[c])
                for w_idx in word_indices:
                    class_scores[c] += np.log(word_probs[c][vocabulary[w_idx]])
            predicted_labels.append(max(class_scores, key=class_scores.get))
        return predicted_labels
        # END CODE
        # END CODE

nb = NaiveBayesClassifier()
X_text = ["hello world", "beautiful day"]
print(nb.predict(X_text))
# vocab = nb.create_vocabulary(texts)
# array = nb.extract_features(texts, vocab)
# print(vocab)
# print(array)
# class_probs = nb.calculate_class_probabilities([0, 0, 1, 1, 0, 1])
# print(class_probs)

KeyError: 0